In [1]:
'''
This code generates the orbit file for quantum satellite communication
Last Modified: 2021-09-07
Author: Abhijeet Alase, Karabee Batta, Ziheng Chang, Daniel Oblak

Variable inputs:
sat_num: the satellite number of interest
gs: location of the ground station
t0, t1: defines the time range that we look for satellite passes
ti, tf: defines the start time and end time of one satellite pass (obtain this info from the previous step)

'''

'\nThis code generates the orbit file for quantum satellite communication\nLast Modified: 2021-09-07\nAuthor: Abhijeet Alase, Karabee Batta, Ziheng Chang, Daniel Oblak\n\nVariable inputs:\nsat_num: the satellite number of interest\ngs: location of the ground station\nt0, t1: defines the time range that we look for satellite passes\nti, tf: defines the start time and end time of one satellite pass (obtain this info from the previous step)\n\n'

In [2]:
#Import functions and libraries
from skyfield.api import load, wgs84
from skyfield.api import EarthSatellite
from datetime import timedelta
import numpy as np

In [3]:
sat_num = 41731 #satellite number for Micius
gs = wgs84.latlon(+51.049999, -114.066666) #location of Calgary ground station
gs = wgs84.latlon(+51.078621, -114.136719) #location of ground station (University of Calgary)  

ts = load.timescale()
stations_url = 'http://celestrak.com/NORAD/elements/active.txt'
satellites = load.tle_file(stations_url)
by_number = {sat.model.satnum: sat for sat in satellites}
satellite = by_number[sat_num]
print(satellite)

difference = satellite - gs

QSS (MOZI) catalog #41731 epoch 2021-09-06 20:15:12 UTC


In [4]:
#Condition of the satellite now
t = ts.now()
geocentric = satellite.at(t)
print(geocentric.position.km)

subpoint = wgs84.subpoint(geocentric)
topocentric = difference.at(t)
print(topocentric.position.km)
alt, az, distance = topocentric.altaz()

if alt.degrees > 0:
    print('The ISS is above the horizon')

print('Altitude: {:.6f} deg'.format(alt.degrees))
print('Azimuth: {:.6f} deg'.format(az.degrees))
print('Distance: {:.1f} km'.format(distance.km))
print('Height: {:.1f} km'.format(subpoint.elevation.km))

[4495.88673827 -837.48853402 5117.60907811]
[605.92723393 196.81874565 186.62025806]
The ISS is above the horizon
Altitude: 46.545425 deg
Azimuth: 130.697647 deg
Distance: 663.9 km
Height: 497.1 km


In [5]:
#Find pass times for a certain time range
t0 = ts.utc(2021,9,4) #start time
t1 = ts.utc(2021,9,5) #end time
t, events = satellite.find_events(gs, t0, t1, altitude_degrees=15.0)
for ti, event in zip(t, events):
    name = ('Reaches altitude 15°', 'Maximum altitude', 'Drops below altitude 15°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

2021 Sep 04 07:04:18 Reaches altitude 15°
2021 Sep 04 07:06:39 Maximum altitude
2021 Sep 04 07:08:58 Drops below altitude 15°
2021 Sep 04 08:37:51 Reaches altitude 15°
2021 Sep 04 08:40:09 Maximum altitude
2021 Sep 04 08:42:25 Drops below altitude 15°
2021 Sep 04 17:45:45 Reaches altitude 15°
2021 Sep 04 17:47:57 Maximum altitude
2021 Sep 04 17:50:10 Drops below altitude 15°
2021 Sep 04 19:18:57 Reaches altitude 15°
2021 Sep 04 19:21:24 Maximum altitude
2021 Sep 04 19:23:53 Drops below altitude 15°


In [6]:
#Initialization
ti = ts.utc(2021, 9, 4, 8, 37, 52) #start time
tf = ts.utc(2021, 9, 4, 8, 42, 25) #end time
n = 10000 #max time steps (just a large number, useful for testing purposes)
i = 0 #initialize index
t_step = ti #initialize time step
result = [1,1,1,1,1,1,1,1,1] #initialize the result matrix (this first line will be deleted)

In [7]:
#Loop to generate result matrix (main part of code)
while i < n:
    #Calculate needed values
    geocentric = satellite.at(t_step)
    subpoint = wgs84.subpoint(geocentric)
    topocentric = difference.at(t_step)
    alt, az, distance = topocentric.altaz()
    #Input the data to the matrix
    new_data = [sat_num, int(t_step.utc_strftime('%Y%m%d')), int(t_step.utc_strftime('%H%M%S000')), az.degrees,
            0, alt.degrees, 0, distance.km, subpoint.elevation.km]
    result = np.vstack((result,new_data))
    #Advance the time step by 1 second
    t_step = ts.utc(t_step.utc_datetime() + timedelta(seconds=1))
    #If end time is reached, finish the loop
    if t_step == tf:
        break
    #Increment the index
    i = i + 1

In [8]:
#Output
result = np.delete(result, (0), axis=0) #delete the first line
fmt = ['%05d', '%08d', '%09d', '%1.6f', '%1.6f', '%1.6f', '%1.6f', '%1.6f', '%1.2f'] #set the required format
title = int(ti.utc_strftime('%Y%m%d')) #yyyymmdd for title
np.savetxt('X{0}_Cal.dat'.format(title), result, fmt=fmt) #output the .dat orbit file